In [ ]:
%pip install robobrowser

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Heman\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [ ]:
# Python 2-3 compatibility
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)
from builtins import *

In [ ]:
import requests
from getpass import getpass
from time import sleep
import json
import re

In [ ]:
import tinder_api as api
import phone_auth

In [ ]:
def format_phone(number):
    """Takes a string, strips the non-digits, and checks to see that it is international format."""
    # number = str(number)
    # number = re.sub(r'\D', '', number)  # remove non-digits with empy char
    # if len(number) > 13 or len(number) < 11:
    #     raise ValueError('Number must be in international format!')
    number=447508862220
    return number

In [ ]:
print('Enter your phone number for the Tinder account, in international format. '
      'Be sure that you have already set up the account and verified the number and email on Tinder!')

# phone_num = format_phone(input("phone number:"))
phone_num = int("00447508862220")

req_code = phone_auth.sendCode(phone_num)
if req_code is False:
    raise ValueError('There was an error when sending the phone number!')
else:
    print('Your request code is:', req_code)

sms_tries = 0
sms_success = False
for sms_tries in range(3):
        
    sms_code = input("Please enter the code you've received by sms: ")
    tinder_token = phone_auth.getToken(phone_num, sms_code, req_code)
    if tinder_token is not None:
        sms_success = True
        api.headers.update({"X-Auth-Token": tinder_token})
        break

if not sms_success:
    raise ValueError("SMS token incorrectly entered 3 times, run the cell again")
    
print("Tinder token:", tinder_token)

Enter your phone number for the Tinder account, in international format. Be sure that you have already set up the account and verified the number and email on Tinder!


ValueError: Number must be in international format!

In [ ]:
def get_name(recs_result):
    """Helper function to take ain item of the recommended users and convert them to a name."""
    return recs_result['user']['name'].lower()

In [ ]:
def find_people(names, num_to_search, delay=0.5):
    """Find people on tinder by name
    
    Args:
        names: A list of strings to search for
        num_to_search: The minimum number of people to search through
        delay: The time in seconds between each request to tinder's servers.
        
    Returns: A tuple with a list of python dicts for each user that matched a name in `names`,
        and the number of people searched through.
    """
    if isinstance(names, str):
        raise ValueError("`names` should be an iterable!")
    names = set(n.lower() for n in names)
    count = 0
    results = []
    while count < num_to_search:
        try:
            response = requests.get(api.host+'/v2/recs/core?locale=en-US', headers=api.headers)
            if not response.ok:
                break
            print(response.json())
            recs = response.json()["data"]["results"]

            for rec in recs:
                rec_name = get_name(rec)
                if rec_name in names:
                    results.append(rec)
            count += len(recs)
            sleep(delay)
        except Exception as e:
            print(e)
        
    return results, count

In [ ]:
def get_link_from_id(id):
    """Gets the sharable link for someone's profile. To view you must have the app. 
    Limited uses for each generated link."""
    response = requests.post(api.host + '/user/%s/share' % id, headers=api.headers)
    if not response.ok:
        return False
    return response.json()['link']

In [ ]:
def set_loc(lat, lon):
    """Returns whether the location was updated was successfully."""
    data = {'lat':lat, 'lon':lon}
    response = requests.post(api.host + '/user/ping', headers=api.headers, data=json.dumps(data))
    return response.ok

In [ ]:
set_loc(42.4439614, -76.5018807)  # NYC

In [ ]:
# this is easier to do in the app, but in case you want to do it from python, here it is
api.change_preferences(age_filter_max=22, age_filter_min=24, distance_filter=5)

In [ ]:
results, num_searched = find_people(['sarah'], 100)
results, num_searched

In [ ]:
# extract the user ids and sharable links from the results for future reference and easier viewing
for r in results:
    _id = r['user']['_id']
    print('id:', _id, 'link:', get_link_from_id(_id))

In [ ]:
get_link_from_id('5bf6d6d99c8a60df4b6c721c')